In [3]:
#Setup
import pandas as pd
import os

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

<p style="font-family: Open Sans; color: #2B6264; font-weight: bold; font-size: 125%">What are we going to cover?</p>

<p style="font-family: Open Sans; color: #2B6264">- How can we create a LUSID webhook notification using the Notifications API via the Python SDK?</p>

<p style="font-family: Open Sans; color: #2B6264">- What are our options for content type, authentication etc. when sending webhook notifications?</p>

<p style="font-family: Open Sans; color: #2B6264">- How can we create a 3rd party webhook notification using the Notifications API via the Python SDK?</p>

<p style="font-family: Open Sans; color: #2B6264; font-weight: bold; font-size: 125%">Creating 3rd Party Webhook Notification - Setup</p>

In [ ]:
import lusid_configuration
from lusidjam import RefreshingToken
from lusid_configuration.extensions import (
    SyncApiClientFactory,
    ArgsConfigurationLoader,
    EnvironmentVariablesConfigurationLoader,
    SecretsFileConfigurationLoader
)
from lusid_configuration.models import *

# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")
api_url_configuration = os.getenv("FBN_CONFIGURATION_API_URL")
# Run the Notebook locally using a secrets file (see https://support.lusid.com/docs/how-do-i-use-an-api-access-token-with-the-lusid-sdk)
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
config_loaders_configuration=[
    ArgsConfigurationLoader(access_token = RefreshingToken(), app_name = "LusidJupyterNotebook", api_url=api_url_configuration),
    EnvironmentVariablesConfigurationLoader(),
    SecretsFileConfigurationLoader(secrets_path)]
api_factory_configuration = lusid_configuration.extensions.SyncApiClientFactory(config_loaders=config_loaders_configuration)

configuration_sets_api = api_factory_configuration.build(lusid_configuration.api.ConfigurationSetsApi)
display(configuration_sets_api)

<p style="font-family: Open Sans; color: #2B6264; font-weight: bold; font-size: 125%">Creating a LUSID Webhook Notification</p>

In [ ]:
import pprint

import lusid_notifications
from lusidjam import RefreshingToken
from lusid_notifications.extensions import (
    SyncApiClientFactory,
    ArgsConfigurationLoader,
    EnvironmentVariablesConfigurationLoader,
    SecretsFileConfigurationLoader
)
from lusid_notifications.models import *

api_url_notifications = os.getenv("FBN_NOTIFICATIONS_API_URL")

config_loaders_notifications=[
    ArgsConfigurationLoader(access_token = RefreshingToken(), app_name = "LusidJupyterNotebook", api_url=api_url_notifications),
    EnvironmentVariablesConfigurationLoader(),
    SecretsFileConfigurationLoader(secrets_path)]
api_factory_notifications = lusid_notifications.extensions.SyncApiClientFactory(config_loaders=config_loaders_notifications)

notifications_api = api_factory_notifications.build(lusid_notifications.api.NotificationsApi)
subscriptions_api = api_factory_notifications.build(lusid_notifications.api.SubscriptionsApi)
display(notifications_api)
display(subscriptions_api)


subscription_scope = "FinbourneUniversity"
subscription_code = "AnyPortfolioCreatedv2"

In [6]:
try:
    subscriptions_api.get_subscription(
        scope=subscription_scope,
        code=subscription_code)
    
    subscriptions_api.delete_subscription(
        scope=subscription_scope,
        code=subscription_code
    )
    
except lusid_notifications.ApiException as api_exception:
    if api_exception.status == 404:
        pass
    else:
        raise

In [ ]:
subscriptions_api.create_subscription(
    create_subscription=lusid_notifications.CreateSubscription(
        id=lusid_notifications.ResourceId(
            scope=subscription_scope,
            code=subscription_code,
        ),
        display_name="Any Portfolio Created",
        description="Listens to all TransactionPortfolioCreated events",
        status="Active",
        matching_pattern=lusid_notifications.MatchingPattern(
            event_type="PortfolioCreated",
            filter="Body.portfolioScope eq 'FinbourneUniversity' and Body.portfolioCode startsWith 'NewTradingPortfolio'"
        )
    )
)

In [ ]:
notifications_api.create_notification(
    scope=subscription_scope,
    code=subscription_code,
    create_notification_request = lusid_notifications.models.CreateNotificationRequest.from_dict(
        {
            "notificationId": "PortfolioCreatedWebhook",
            "displayName": "PortfolioCreatedWebhook",
            "description": "Portfolio created webhook notification",
            "notificationType": {
                "type": "Webhook",
                "httpMethod": "POST",
                "url": "/api/api/transactionportfolios/{{Body.portfolioScope}}/{{Body.portfolioCode}}/transactions",
                "authenticationType": "Lusid",
                "contentType": "Json",
                "content": [
                    {
                        "transactionId": "SeedFundsTransactionUSD",
                        "type": "FundsIn",
                        "instrumentIdentifiers": {
                          "Instrument/default/Currency": "USD",
                        },
                        "transactionDate": "2023-02-01T00:00:00Z",
                        "settlementDate": "2023-02-01T00:00:00Z",
                        "units": 10000000,
                        "transactionPrice": {
                          "price": 1,
                          "type": "Price"
                        },
                        "totalConsideration": {
                          "amount": 10000000,
                          "currency": "USD"
                        },
                        "transactionCurrency": "USD"
                    },
                    {
                        "transactionId": "SeedFundsTransactionGBP",
                        "type": "FundsIn",
                        "instrumentIdentifiers": {
                          "Instrument/default/Currency": "GBP",
                        },
                        "transactionDate": "2023-02-01T00:00:00Z",
                        "settlementDate": "2023-02-01T00:00:00Z",
                        "units": 7500000,
                        "transactionPrice": {
                          "price": 1,
                          "type": "Price"
                        },
                        "totalConsideration": {
                          "amount": 7500000,
                          "currency": "GBP"
                        },
                        "transactionCurrency": "GBP"
                    },
                    {
                        "transactionId": "SeedFundsTransactionAUD",
                        "type": "FundsIn",
                        "instrumentIdentifiers": {
                          "Instrument/default/Currency": "AUD",
                        },
                        "transactionDate": "2023-02-01T00:00:00Z",
                        "settlementDate": "2023-02-01T00:00:00Z",
                        "units": 25000000,
                        "transactionPrice": {
                          "price": 1,
                          "type": "Price"
                        },
                        "totalConsideration": {
                          "amount": 25000000,
                          "currency": "AUD"
                        },
                        "transactionCurrency": "AUD"
                    }
                ]
            }
        }
    )
)

<p style="font-family: Open Sans; color: #2B6264; font-weight: bold; font-size: 125%">Webhook Options</p>

<p style="font-family: Open Sans; color: #2B6264">Content Types</p>
<ul>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">JSON (application/json)</p></li>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">PlainText (text/plain)</p></li>
</ul>

<p style="font-family: Open Sans; color: #2B6264">Methods</p>
<ul>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">POST</p></li>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">PUT</p></li>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">DELETE</p></li>
</ul>

<p style="font-family: Open Sans; color: #2B6264">Authentication Options</p>
<ul>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">Lusid</p></li>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">Bearer</p></li>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">Basic Auth</p></li>
</ul>

In [9]:
try:
    configuration_sets_api.get_configuration_set(
        type="Shared",
        scope="FinbourneUniversity",
        code="jira"
    )
    
    configuration_sets_api.delete_configuration_set(
        type="Shared",
        scope="FinbourneUniversity",
        code="jira"
    )
    
except lusid_configuration.ApiException as api_exception:
    if api_exception.status == 404:
        pass
    else:
        raise

In [10]:
config_response = configuration_sets_api.create_configuration_set(
    create_configuration_set=lusid_configuration.CreateConfigurationSet(
        id=lusid_configuration.ResourceId(
            scope="FinbourneUniversity",
            code="jira"
        ),
        type="Shared",
        description="Access tokens for JIRA"
    )
)

configuration_sets_api.add_configuration_to_set(
    type="Shared",
    scope="FinbourneUniversity",
    code="jira",
    create_configuration_item=lusid_configuration.CreateConfigurationItem(
        key="api-token",
        value="SampleToken",
        value_type="text",
        is_secret=True,
        description="API Key for JIRA"
    )
)

configuration_sets_api.add_configuration_to_set(
    type="Shared",
    scope="FinbourneUniversity",
    code="jira",
    create_configuration_item=lusid_configuration.CreateConfigurationItem(
        key="username",
        value="SampleUsername",
        value_type="text",
        is_secret=False,
        description="Username for JIRA"
    )
)

<p style="font-family: Open Sans; color: #2B6264; font-weight: bold; font-size: 125%">Creating 3rd Party Webhook Notification</p>

In [ ]:
notifications_api.create_notification(
    scope=subscription_scope,
    code=subscription_code,
    create_notification_request = lusid_notifications.models.CreateNotificationRequest.from_dict(
        {
            "notificationId": "PortfolioCreatedJiraWebhook",
            "displayName": "PortfolioCreatedJiraWebhook",
            "description": "New Portfolio Created - Ticket",
            "notificationType": {
                "type": "Webhook",
                "httpMethod": "POST",
                "url": "https://acmecorp.atlassian.net/rest/api/3/issue",
                "authenticationType": "BasicAuth",
                "authenticationConfigurationItemPaths": {
                    "username": "config://shared/FinbourneUniversity/jira/username",
                    "password": "config://shared/FinbourneUniversity/jira/api-token"
                },
                "contentType": "Json",
                "content": {
                    "fields": {
                        "project": {
                            "key": "DEMO"
                        },
                        "issuetype": {
                            "name": "Task"
                        },
                        "summary": "TransactionPortfolio {{body.portfolioCode}} Created!",
                        "description": {
                            "type": "doc",
                            "version": 1,
                            "content": [
                                {
                                "type": "paragraph",
                                "content": [
                                    {
                                    "text": "A portfolio has been created. Please make your first trades!",
                                    "type": "text"
                                    }
                                ]
                                },
                                {
                                "type": "paragraph",
                                "content": [
                                    {
                                    "text": "Please close this ticket once the first trade has been placed",
                                    "type": "text"
                                    }
                                ]
                                }
                            ]
                        },
                        "assignee": {
                            "id": "5bfe63e1ec71bd223bbe623c"
                        },
                        "labels": [
                            "trades",
                            "newPortfolio"
                        ]
                    }
                }
            }
        }
    )
)

In [12]:
subscriptions_api.delete_subscription(
    scope=subscription_scope,
    code=subscription_code
)

configuration_sets_api.delete_configuration_set(
    type="Shared",
    scope="FinbourneUniversity",
    code="jira"
)

<p style="font-family: Open Sans; color: #2B6264; font-weight: bold; font-size: 125%">What have we covered?</p>

<p style="font-family: Open Sans; color: #2B6264">- We created a LUSID webhook notification using the Notifications API via the Python SDK.</p>

<p style="font-family: Open Sans; color: #2B6264">- We looked at the available content types, authentication methods and request types for webhook notifications.</p>

<p style="font-family: Open Sans; color: #2B6264">- We created a 3rd party webhook notification using the Notifications API via the Python SDK.</p>